In [65]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [66]:
df_clientes = pd.read_csv('./Datasets/Clientes.csv', delimiter=';', encoding='utf-8')
df_compra = pd.read_csv('./Datasets/Compra.csv')
df_gasto = pd.read_csv('./Datasets/Gasto.csv')
df_local = pd.read_csv('./Datasets/Localidades.csv')
df_prov = pd.read_csv('./Datasets/Proveedores.csv', encoding='iso-8859-1')
df_suc = pd.read_csv('./Datasets/Sucursales.csv', delimiter=';')
df_venta = pd.read_csv('./Datasets/Venta.csv')

In [67]:
df_venta.describe()

,IdVenta,IdCanal,IdCliente,IdSucursal,IdEmpleado,IdProducto,Precio,Cantidad
count,46645.000000,46645.000000,46645.000000,46645.000000,46645.000000,46645.000000,4.572100e+04,45753.000000
mean,23398.589624,2.016958,1502.721514,14.781927,2488.720956,42901.305585,5.835634e+03,2.385395
std,13590.352830,0.743113,972.521043,8.878188,960.499762,83.563413,2.498016e+05,3.547991
min,1.000000,1.000000,1.000000,1.000000,1011.000000,42737.000000,3.000000e+00,1.000000
25%,11662.000000,1.000000,692.000000,7.000000,1574.000000,42833.000000,4.380000e+02,2.000000
50%,23323.000000,2.000000,1360.000000,15.000000,2468.000000,42902.000000,6.536200e+02,2.000000
75%,34984.000000,3.000000,2273.000000,23.000000,3433.000000,42974.000000,1.789000e+03,3.000000
max,48241.000000,3.000000,3407.000000,31.000000,3979.000000,43043.000000,3.373920e+07,300.000000


In [68]:
productos = [x for x in df_venta['IdProducto'].unique()]
print(productos)

[42817, 42795, 42837, 42834, 42825, 42852, 42939, 42940, 42905, 42894, 42998, 42969, 42960, 42987, 43010, 43015, 42976, 42958, 42992, 42797, 42789, 42766, 42787, 42763, 42754, 42876, 42830, 42879, 42861, 42859, 42945, 42906, 42893, 43016, 42970, 42975, 43033, 43022, 43026, 43017, 42758, 42785, 42800, 42796, 42764, 42814, 42765, 42737, 42776, 42788, 42799, 42774, 42760, 42816, 42815, 42793, 42810, 42782, 42783, 42772, 42762, 42779, 42790, 42818, 42777, 42791, 42884, 42846, 42839, 42831, 42840, 42848, 42849, 42833, 42845, 42841, 42865, 42836, 42871, 42858, 42844, 42883, 42869, 42824, 42822, 42870, 42862, 42878, 42820, 42855, 42873, 42827, 42914, 42885, 42949, 42903, 42947, 42913, 42929, 42934, 42923, 42902, 42907, 42948, 42941, 42896, 42916, 42930, 42931, 42888, 42927, 42938, 42912, 42891, 42920, 42886, 42909, 42890, 43013, 42973, 43006, 42955, 42968, 42951, 43008, 42988, 42964, 42995, 42990, 42994, 42986, 43004, 42977, 42953, 43003, 42956, 43001, 43012, 43009, 42984, 43011, 42972, 42963

In [69]:
def get_moda(df, element_id, referencia, target):
    """Obtiene la moda de una columna target filtrando a través de la columna referencia"""
    df_moda = df[df[referencia] == element_id].mode()
    return df_moda[target][0]

In [70]:
def input_moda(df, element_id, referencia, target):
    """Cambia todos los valores de las columnas al valor de la moda"""
    moda = get_moda(df, element_id, referencia, target)
    df.loc[(df[target] != moda) & (df[referencia] == element_id), target] = moda

In [74]:
for product in productos:
    input_moda(df_venta, product, 'IdProducto', 'Precio')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46645 entries, 0 to 46644
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   IdVenta        46645 non-null  int64  
 1   Fecha          46645 non-null  object 
 2   Fecha_Entrega  46645 non-null  object 
 3   IdCanal        46645 non-null  int64  
 4   IdCliente      46645 non-null  int64  
 5   IdSucursal     46645 non-null  int64  
 6   IdEmpleado     46645 non-null  int64  
 7   IdProducto     46645 non-null  int64  
 8   Precio         46645 non-null  float64
 9   Cantidad       45753 non-null  float64
dtypes: float64(2), int64(6), object(2)
memory usage: 3.6+ MB
